<a href="https://colab.research.google.com/github/Kalabras/Taiwan-HousePrice-Prediction/blob/main/Taiwan_House_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Modele pour predicter les prix des maisons en Taiwan**
par Abderrahmen Jbeli et Wala Khedri

Etape 1: Importation des librairies neccesaire:

> Les librairies utilisé:
>>_Pandas: Librairy populaire pour travailler avec les donneés.
>>
>>_Numpy: Library populaire qui contient des fonctions mathématique avancé pour travailler avec les donneés.
>>
>>_Matplotlib: Pour Visualiser les donneés.
>>
>>_SKLearn: Library pour travailler dans l'intelligance artificielle, poccéde plusieur outils trés important que nous utilisons comme *train_test_split* , *StandradScaler* , *LinearRegression*, *mean_squared_error* , *r2_score* et plusieur autre.
>>
>>_Joblib: Library utilisé pour sauvegarder et charger les modéles

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from joblib import dump,load

Etape 2: Charger les donneé (data)

On charge les donneé de type *.csv* avec la fonction *read_csv* de Pandas

In [7]:
tai_hou = pd.read_csv("/content/maisons_taiwan.csv") # On utilise la fonction read_csv() du library Pandas pour lire un fichier .csv qui contient nos donneés

In [8]:
tai_hou.describe() # on utilise describe pour avoir des information sur nos donneés

,date,age,metro,epicerie,latitude,longitude,prix
count,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000
mean,2013.148971,17.712560,1083.885689,4.094203,24.969030,121.533361,37.980193
std,0.281967,11.392485,1262.109595,2.945562,0.012410,0.015347,13.606488
min,2012.667000,0.000000,23.382840,0.000000,24.932070,121.473530,7.600000
25%,2012.917000,9.025000,289.324800,1.000000,24.963000,121.528085,27.700000
50%,2013.167000,16.100000,492.231300,4.000000,24.971100,121.538630,38.450000
75%,2013.417000,28.150000,1454.279000,6.000000,24.977455,121.543305,46.600000
max,2013.583000,43.800000,6488.021000,10.000000,25.014590,121.566270,117.500000


In [9]:
tai_hou.head() # on utilise head() pour voir les 5 premiers lignes de nos donneés

,date,age,metro,epicerie,latitude,longitude,prix
0,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1


In [10]:
tai_hou.shape # shape retourne combien de ligne et colonne existe dans les donneé

(414, 7)

In [11]:
tai_hou.columns # retourne les noms des colonnes dans les donneés

Index(['date', 'age', 'metro', 'epicerie', 'latitude', 'longitude', 'prix'], dtype='object')

In [12]:
tai_hou.isnull().sum() # retourne le nombre des lignes vides dans chaque colonne

date         0
age          0
metro        0
epicerie     0
latitude     0
longitude    0
prix         0
dtype: int64

Etape 3: Nettoyer notre data:

Il faut nettoyer notre data de tous imperfections comme les lignes ou colonnes vides ou les colonnes qui siginifie rien.

Dans notre example on efface la colonne date car on ne va pas la utilisé

In [13]:
tai_hou.drop(columns="date",inplace=True) # Effacer la colonne "date"

Etape 5: On divise notre data sur 4 espaces geographique

On peut faire ca en translater les cordonner de longitude et latitude en binaire, on calcule le moyenne des longitude et le moyenne des latitude et assigner 1 pour les cordonner supérieur au moyenne et 0 pour les cordonner inférieur au moyenne

In [14]:
def binariser(data, column): # declaration de fonction "binariser"
  moy = data[column].mean() # calcule du moyenne
  print(moy)
  data[column] = (data[column]>moy).astype(float) # assigner 1 et 0 par comparaison de chaque valeur avec le moyenne

In [15]:
binariser(tai_hou,"longitude") # binariser les valeurs de longitude
binariser(tai_hou, "latitude") # binariser les valeurs de latitude

121.53336108695655
24.969030072463767


In [16]:
tai_hou.head()

,age,metro,epicerie,latitude,longitude,prix
0,32.0,84.87882,10,1.0,1.0,37.9
1,19.5,306.59470,9,1.0,1.0,42.2
2,13.3,561.98450,5,1.0,1.0,47.3
3,13.3,561.98450,5,1.0,1.0,54.8
4,5.0,390.56840,5,1.0,1.0,43.1


Etape 6: selection des donneés features et donneé a predicter:

Ici on selectionne les donneés characteristique et les donneés cible que le modéle va predicter.

Dans notre modéle, le cible et la colonne "Prix" et toute autre colonne sont des characteristiques

In [17]:
X = tai_hou.iloc[:,:-1] # On selctionne tout les lignes et tout les colonnes sauf la dérniere pour les characteristiques
y = tai_hou.iloc[:,-1] # On selectionne tout les lignes et seulement la dérniére colonne comme la cible

In [18]:
X.head() # affichage des charactéristiques

,age,metro,epicerie,latitude,longitude
0,32.0,84.87882,10,1.0,1.0
1,19.5,306.59470,9,1.0,1.0
2,13.3,561.98450,5,1.0,1.0
3,13.3,561.98450,5,1.0,1.0
4,5.0,390.56840,5,1.0,1.0


In [19]:
y.head() # affichage des cibles

0    37.9
1    42.2
2    47.3
3    54.8
4    43.1
Name: prix, dtype: float64

Etape 7: On devise nos donneé en donneé d'entrainment et donneé de test:

Ici on devise nos donneé en donneé d'entrainment que le modéle va étudier avec et des donneés de test qu'on va tester l'efficacité de notre modéle avec.

In [29]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)  


# On utilise train_test_split pour deviser nos donneé en 4 partie:
# 
# * X_train: ce sont les charactéristiques que le modéle va s'entrainer avec.
# * y_train: ce sont les réponces de cible (Prix) que le modéle va comparer au charactéristique (X_train) au cours de l'entrainment.
# * X_test: ce sont les charactéristique qu'on va présenter au modéle aprés sont entrainment pour l'evaluer.
# * y_test: ce sont les réponces de cible (Prix) des charactéristiques (X_test) qu'on va comparer avec les prédictions fait par le modéle pour mesurer
#    sa présision.


Etape 7: Standarisation:

Ici on standarise les valeurs qu'on a pour avoir un moyenne de 0 et une déviation de 1 dans chaque sens, c-a-d tout les valeurs vont étre entre -1 et 1 avec le moyenne 0, on fait ca pour mettre tout les valeurs le plus petit que possible car ca facilite l'entrainment du modéle.

In [21]:
sc_X = StandardScaler()  # On creé un object "Scaler" avec la fonction "StandardScaler()" de library sklearn.preprocessing
X_train = sc_X.fit_transform(X_train) # On utilise cet objet pour standariser les valeurs d'entraiment X_train
X_test = sc_X.fit_transform(X_test)  # On standarise les valeurs de test aussi

Etape 8: **L'ENTRAINMENT DU MODELE**

Ici on entraine notre Modéle avec les charactéristiques X_train et cet correspondant cible y_train, le modéle va comparer les valeurs et générer une coefficient pour chaque charactéristique pour déduire le prix!!

In [22]:
modele = LinearRegression().fit(X_train,y_train)  # On utilise l'algorithme "Linear Regression" pour entrainer notre modéle
# On utilise la fonction LinearRegression().fit() de la library sklearn.linear_model

In [23]:
print("Coefficients : ",modele.coef_) # On empreinte les coefficients générer par le modéle

Coefficients :  [-3.6919379  -4.73701178  3.48632549  3.13637857  0.98929164]


Etape 9: La Prédiction

Ici on utilise notre modéle entrainer pour prédicter les valeur des prix dans la partie du test de nos donneés (X_test)

In [24]:
y_pred = modele.predict(X_test) # On utilise la fonction predict() avec les donneés X_test pour prédicter les prix et on les sauvegarde dans y_pred

Etape 10: Calcule d'erreur

Ici on compare les résultat obtenue par le modéle avec les vrais prix pour connaitre le précision de notre modéle.

On utilise l'algorithme Mean Squared Error pour avoir l'erreur dans les prédictions de notre modéle, le plus faible l'erreur et mailleur.

Aussi on calcule le score R2 du notre modéle, c'est la coefficient de détérmination qui décrit a quelle degré notre modéle prédit bien la variation dans notre cible, elle est entre 0 et 1 avec 1 étant un modéle parfait.

In [25]:
print("Error: ",mean_squared_error(y_test,y_pred)) # Calcule d'erreur avec la fonction mean_squared_error de la library sklearn.metrics
print("R2 Score: ",r2_score(y_test,y_pred)) # Calcule de coefficient de détérmination (R2 Score) avec la fonction r2_score de la library sklearn.metrics

Error:  60.27210019737884
R2 Score:  0.607941066894341


Etape 11: Enregistrement du modéle

Ici on sauvegarde notre modéle pour l'utilisé ailleur, aussi pour l'utiliser sans le retrainer chaque fois, car l'entrainement du notre modéle prends beaucoup du temps et du resources (depend a l'échelle de nos data).

In [26]:
dump(modele,"Taiwan_House_Predictions.joblib") # On utilise la fonction dump() du library joblib pour sauvegarder notre modéle en un fichier .joblib

['Taiwan_House_Predictions.joblib']

Etape 12: Utilisation de notre modéle

La fabrication du modéle est terminer, ici on importe notre modéle depuis le fichier .joblib qu'on a sauvegarder a l'étape précédente pour l'utiliser a prédicter de noveau data!!

In [27]:
modele_use = load("Taiwan_House_Predictions.joblib") # On utilise la fonction load() du library joblib pour charger notre modéle d'une fichier .joblib

In [28]:
y_pred2 = modele_use.predict(X_test) # Finallment, on utilise le modéle chargé pour faire de noveau prédiction

#***Fin***. 